# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("../data/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
            'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all s

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,18.295090,2.113620,0.305677,0.074934,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.974865,0.975741,0.969452,0.972147,0.973953,0.002848,1
8,33.346648,0.366018,0.549872,0.059144,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.976661,0.975741,0.967655,0.969452,0.973593,0.004245,2
11,24.359629,0.576179,0.232491,0.020892,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.977558,0.973046,0.967655,0.972147,0.973413,0.003539,3
7,14.934324,0.407502,0.349656,0.094491,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.973968,0.974843,0.967655,0.971249,0.973055,0.003368,4
4,14.207083,0.571385,0.334738,0.088223,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.978456,0.973968,0.972147,0.967655,0.971249,0.972695,0.003538,5


In [4]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
            'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hu3745fi/opt/anaconda3/envs/nlp_linkedin/lib/python3.9/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all s

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,25.134968,0.318471,0.238420,0.023443,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.972172,0.973944,0.969452,0.972147,0.973055,0.002670,1
10,15.819694,0.864151,0.251180,0.017459,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.973968,0.974843,0.967655,0.971249,0.973055,0.003368,2
8,29.073140,0.212784,0.386870,0.035504,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.973070,0.973944,0.966757,0.971249,0.972695,0.003801,3
7,14.542484,0.322097,0.330609,0.127461,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.973968,0.974843,0.967655,0.967655,0.972156,0.003777,4
3,1.738480,0.315252,0.155043,0.036830,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.982047,0.973968,0.966757,0.964061,0.973046,0.971976,0.006268,5
